# Рисуем интерактивные карты с Folium

В питоне есть множество библиотек, с помощью которых можно рисовать и анализировать пространственную информацию (spatial analysis).

Вот некоторые из них:

* folium
* gmaps
* basemap
* cartopy
* geoplotlib


В этом туториале пойдет речь о библиотеке Folium (https://github.com/python-visualization/folium), которая представляет собой питон-обертку над JS библиотекой Leaflet.

"Manipulate your data in Python, then visualize it in on a Leaflet map via Folium." (c) github

Большим плюсом по сравнению с другими библиотеками является интерактивность. Карты можно зумить, исследовать, кликать на маркеры, создать сложные типы визуализации.
Минусом ялвяется производительность. Создание карты с большим количеством точек может составлять минуты.

К сожалению, информация по работе с библиотекой раскидана по разным сайтам и туториалам. В официальной документации информации очень мало. А в русскоязычном интернете вообще ничего нет.

Будем исследовать датасет с пабами Москвы =). Идея нагло украдена отсюда:

http://www.math.uwaterloo.ca/tsp/pubs/

В конце туторила так же попробуем отобразить кратчайший путь через пабы Москвы.

## Подготовка данных

In [234]:
import json
import pandas as pd
import folium
import requests

with open('pubs.json') as json_data:
    d = json.load(json_data)
    
columns = ['lat', 'lon', 'name_ru', 'opening_hours', 'website']
index = range(0, len(d["data"]))
pubs = pd.DataFrame(columns = columns, index = index)

for i in range(0, len(d["data"])):
    pubs['lat'].iloc[i] = d["data"][i]["lat"]
    pubs['lon'].iloc[i] = d["data"][i]["lon"]
    pubs['opening_hours'].iloc[i] = d["data"][i]["opening_hours"]
    pubs['website'].iloc[i] = d["data"][i]["website"]
    pubs['name_ru'].iloc[i] = d["data"][i]["name_ru"]    

In [235]:
pubs.head(3)

,lat,lon,name_ru,opening_hours,website
0,55.6051,37.3538,Московский дворик,None,None
1,55.9092,37.8661,Кружка,None,None
2,55.8649,37.6056,Кружка,None,http://www.kruzhka.ru/


In [236]:
# Для центрирования карт я выбрала центральную точку Москвы

kreml = [55.750730, 37.617322]

## Dot density map

Самый простой вариант анализа - отобразить данные как точки (или маркеры) на карте. В popup (выноску) положим название заведения и часы работы.

Карта инициализируется с помощью синтаксиса `map = folium.Map(location=kreml, zoom_start=11)`.

Добавляем маркеры `folium.Marker().add_to(map)`.

Различные типы маркеров задаются фукнциями:

- Marker (использую для визуализации ниже)
- RegularPolygonMarker
- CircleMarker
- PolygonMarker

Возможные атрибуты: 

- color
- fill_color
- weight
- radius
- number_of_sides (для RegularPolygonMarker)

Так же в выноску можно передавать график vincent (https://github.com/wrobstory/vincent) с помощью синтаксиса: 

`folium.Popup().add_child(folium.Vega())`

`tiles` - стиль карт. Я обычно использую стиль openstreetmaps по умолчанию.

In [237]:
pubs_map = folium.Map(location=kreml, zoom_start=11)

for i in range(0, len(pubs)):
    folium.Marker([pubs['lat'].iloc[i], pubs['lon'].iloc[i]], popup = str(pubs['name_ru'].iloc[i]) + ": " 
                  + str(pubs['opening_hours'].iloc[i])).add_to(pubs_map)
    
pubs_map

Cluster marker - раскраска в зависимости от плотности точек. Близкие точки сливаются в один маркер.

Судя по данным наибольшая плотность пабов в Москве на серево-востоке от Кремля в районе Чистых прудов.

In [238]:
from folium import features

pubs_map = folium.Map(location=kreml, zoom_start=12)
mc = features.MarkerCluster()

for i in range(0, len(pubs)):
    mk = features.Marker([pubs['lat'].iloc[i], pubs['lon'].iloc[i]])
    mc.add_child(mk)
    
pubs_map.add_child(mc)

## Heatmap

Искусственный пример создания heat map.

In [245]:
from folium import plugins
import numpy as np
import random

# с помощью рандома создаю оценки заведений. можете доработать датасет и найти реальные скопления хороших пабов.
pubs['star'] = [int(6*random.random()) for i in range(len(pubs))]

pubs_map = folium.Map(location=kreml, zoom_start=10)

data = [[x[0], x[1], x[2]] for x in np.array(pubs[['lat', 'lon', 'star']])]

HeatMap(data, radius = 20).add_to(pubs_map)
pubs_map

## Линии

Добавим на карту линию, соединящую локации. Случайным образом выбраны 4 заведения в Сокольниках.

In [246]:
soloklniki = [55.791981, 37.664456]

pubs_map_sokolniki = folium.Map(location=soloklniki, zoom_start=13)
path  = []
for i in [100, 101, 102, 103]:
    folium.Marker([pubs['lat'].iloc[i], pubs['lon'].iloc[i]], popup = str(pubs['name_ru'].iloc[i]) + ": " 
                  + str(pubs['opening_hours'].iloc[i])).add_to(pubs_map_sokolniki)
    path.append([[pubs['lat'].iloc[i], pubs['lon'].iloc[i]], [pubs['lat'].iloc[i+1], pubs['lon'].iloc[i+1]]])

folium.PolyLine(path[0:3], color='blue', weight=4, opacity=0.7, popup=str(i)).add_to(pubs_map_sokolniki)
pubs_map_sokolniki   

## Найдем и отобразим кратчайший путь через пабы Москвы :)

Для уменьшения размера выборки выбираю только пабы с наивысшими оценками.

Для нахождения кратчайшего пути использую библиотеку Google or-tools, которая включает в себя алгоритмы решения задач нахождения маршрута. Работа с ней это тема для отдельного туториала, поэтому загружаю найденное решение из внешнего файла.

In [244]:
# to_csv (pubs_final = pubs[pubs['star'] == 5])

,lat,lon,name_ru,opening_hours,website,star
4,55.6595,37.7524,Кружка,Mo-Fr 12:00-00:00; Sa-Su 12:00-04:00,http://www.kruzhka.ru/,5
7,55.6512,37.7445,Кружка,None,http://www.kruzhka.ru/,5
19,55.8599,37.659,Бирлайн,None,None,5
24,55.8697,37.677,Тройка,None,None,5
25,55.6314,37.5189,Чёрная каракатица,None,None,5
39,55.6421,37.5239,Дранкен Дак Паб,None,None,5
48,55.8271,37.4474,Пивной клуб Крокодил,None,None,5
52,55.8157,37.6395,Бирхаус,12:00-23:00,None,5
53,55.8448,37.3504,Bastards,None,None,5
56,55.8992,37.5859,"Разливное пиво ""Лит.Ра""",None,http://www.litra-beer.ru/,5


Have fun =)